# WeatherPy - Itinerary Creation

### Dependencies and data

In [1]:
%matplotlib inline

import datetime as dt
import gmaps
import requests
import numpy as np
import pandas as pd

import config

In [2]:
# Settings
gmaps.configure(api_key=config.GMAPS_API_KEY)

In [3]:
# Load data
hotels = pd.read_csv('data/hotels.csv')
hotels.head(2)

,ID,City,Country,Time,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain,Snow,Hotel
0,1,Vendas Novas,PT,2020-11-05 13:21:54,38.68,-8.46,63.0,59,20,14.99,few clouds,0.0,0.0,Hotel Acez
1,3,Makakilo City,US,2020-11-05 13:21:55,21.35,-158.09,77.0,94,90,3.36,overcast clouds,0.0,0.0,Marriott's Ko Olina Beach Club


### Narrow search and find 4-5 nearby cities

In [4]:
min_mt, max_mt = hotels['Max Temp'].min(), hotels['Max Temp'].max()
min_mt, max_mt

(60.01, 79.99)

In [8]:
def take_input(prompt, min_temp=min_mt, max_temp=max_mt, yes_no=False):
    
    '''
    Take user input for weather preferences.
    
    Args:
        prompt (str) - input prompt
        min_temp (float) - minimum temperature in data
        max_temp (float) - maximum temperature in data
        yes_no (bool) - weather the input should be "yes" or "no" only
        
    Returns:
        [float or str]
    '''
    
    i = 'input'
    n = i.isnumeric()
    
    if yes_no:
        while i not in ['yes', 'no']:
            i = input(prompt)
        i = 1 if i == 'yes' else 0
    else:
        while not n:
            try:
                i = float(input(prompt))
            except:
                continue
            if min_temp <= i <= max_temp:
                n = True
                
    return i
        
    
# Get preferred temperature range
min_temp = take_input('What is the lowest temperature you would like at your destination? ')
max_temp = take_input('What is the highest temperature you would like at your destination? ', min_temp=min_temp)
rain = take_input('Would you like it to rain at your destination? (yes/no) ', yes_no=True)
snow = take_input('Would you like it to snow at your destination? (yes/no) ', yes_no=True)
min_temp, max_temp, rain, snow

What is the lowest temperature you would like at your destination? 66
What is the highest temperature you would like at your destination? 69
Would you like it to rain at your destination? (yes/no) no
Would you like it to snow at your destination? (yes/no) no


(66.0, 69.0, 0, 0)

In [9]:
# Filter for max temp preference
pref_cities = hotels.loc[(hotels['Max Temp'] >= min_temp) & (hotels['Max Temp'] <= max_temp)]

# Filter for rain preference
if rain:
    pref_cities = pref_cities[pref_cities['Rain'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Rain'] == 0]

# Filter for snow preference
if snow:
    pref_cities = pref_cities[pref_cities['Snow'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Snow'] == 0]
    
pref_cities.shape

(25, 14)

In [14]:
# Request url
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
payload = {
    'key': config.GMAPS_API_KEY,
    'type': 'lodging',
    'radius': 5000
}

# Cities without results
cities404 = []

# Find nearest hotel for each city
for idx, row in pref_cities.iterrows():
    payload['location'] = f'{row["Latitude"]},{row["Longitude"]}'
    try:
        response = requests.get(base_url, params=payload)
        response_json = response.json()
        pref_cities.loc[idx, 'Hotel'] = response_json['results'][0]['name']
    except:
        cities404.append(idx)

print(len(cities404))

0


### Temperature heatmap

In [13]:
# Locations and measurements
locations = pref_cities[['Latitude', 'Longitude']]
weights = pref_cities['Max Temp'].apply(lambda t: max(t, 0))

# Info
hotel_info = '''
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Max Temp}F - {Description}</dd>
</dl>
'''

# Layers
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, point_radius=4, max_intensity=100)
mark_layer = gmaps.marker_layer(locations,
                                info_box_content=[hotel_info.format(**row) for idx, row in pref_cities.iterrows()])

# Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=1.75)
fig.add_layer(heat_layer)
fig.add_layer(mark_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Select cities
cities = ['Reconquista', 'Tapes', 'Chui', 'Rocha', 'Departamento de Maldonado']

### Create travel itinerary

In [23]:
# Extract selected cities
cities_df = pref_cities[pref_cities['City'].isin(cities)]
cities_df

,ID,City,Country,Time,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain,Snow,Hotel
22,65,Chui,UY,2020-11-05 13:22:10,-33.70,-53.46,67.41,63,3,12.33,clear sky,0.0,0.0,Nuevo Hotel Plaza
51,164,Reconquista,AR,2020-11-05 13:27:55,-29.15,-59.65,68.79,44,21,12.46,few clouds,0.0,0.0,Grand Hotel
115,448,Departamento de Maldonado,UY,2020-11-05 13:28:58,-34.67,-54.92,68.00,68,75,10.29,broken clouds,0.0,0.0,La Cimarrona
123,472,Rocha,UY,2020-11-05 13:24:44,-34.48,-54.33,67.84,62,40,11.21,scattered clouds,0.0,0.0,Beleza Pura B&B
184,768,Tapes,BR,2020-11-05 13:30:19,-30.67,-51.40,67.87,69,51,15.61,broken clouds,0.0,0.0,Cabanas Camping Union


In [32]:
# Extract coordinates for each city
cities_df = cities_df.loc[[51, 115, 184, 22, 123]]
locations = cities_df[['Latitude', 'Longitude']].values
locations

array([[-29.15, -59.65],
       [-34.67, -54.92],
       [-30.67, -51.4 ],
       [-33.7 , -53.46],
       [-34.48, -54.33]])

In [43]:
# Info
city_info = '''
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
'''

# # Layers
# mark_layer = gmaps.marker_layer(locations,
#                                 info_box_content=[hotel_info.format(**row) for idx, row in hotels.iterrows()])

# Layers
dir_layer = gmaps.directions_layer(coords[0], coords[1], coords[2:], show_markers=False)
mark_layer = gmaps.marker_layer(locations,
                                info_box_content=[city_info.format(**row) for idx, row in cities_df.iterrows()])

# Map
fig = gmaps.figure(center=(-30.0, -60.0), zoom_level=5)
fig.add_layer(dir_layer)
fig.add_layer(mark_layer)
fig

Figure(layout=FigureLayout(height='420px'))